In [1]:
import os
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
import tensorflow_text as tf_text
from sklearn.model_selection import train_test_split

2024-02-25 15:16:42.366846: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-25 15:16:42.702764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 15:16:42.702812: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 15:16:42.771276: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 15:16:42.914218: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-25 15:16:42.916369: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def get_data(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    lines = [line.split('\t') for line in lines]
    context = np.array([context for _, context, _ in lines]) 
    target = np.array([target for target, _, _ in lines])
    return context, target

In [3]:
filepath = Path("datasets/Languages/heb.txt")
raw_context, raw_target = get_data(filepath)

In [4]:
X_train, X_val, y_train, y_val = train_test_split(raw_context, raw_target, test_size=0.2)

In [5]:
BATCH_SIZE = 64
SHUFFLE_SIZE = len(raw_context)

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)

2024-02-25 15:16:47.400193: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 15:16:47.662981: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
def tf_lower_punct_normalize(text):
    text = tf_text.normalize_utf8(text, "NFKD")
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[^ a-zא-ת.?!,¿]", "")
    text = tf.strings.regex_replace(text, "[.?!,¿]", " \0 ")
    text = tf.strings.strip(text)

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text
    

In [14]:
context_text_processor = keras.layers.TextVectorization(max_tokens=15000 ,standardize=tf_lower_punct_normalize, ragged=True)
target_text_processor = keras.layers.TextVectorization(max_tokens=10000, standardize=tf_lower_punct_normalize, ragged=True)

In [15]:
context_text_processor.adapt(train_dataset.map(lambda context, target: context))
target_text_processor.adapt(train_dataset.map(lambda context, target: target))